In [131]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
import datetime
from sklearn.model_selection import train_test_split

# Preparing for modeling
From the analyzed and preprocessed data, we continue the pre-processing in order to prepare it for the modeling phase

In [29]:
df = pd.read_csv('../data/data2.csv')


In [5]:
df.head()

,action_type,combined_shot_type,game_event_id,game_id,lat,loc_x,loc_y,lon,period,playoffs,...,game_date,matchup,opponent,shot_id,time_remaining,points,season_avg,plus_minus,FGP,month
0,Jump Shot,Jump Shot,12,20000012,34.0443,-157,0,-118.4268,1,0,...,2000-10-31,away,POR,2,622,8,18.26,-10.26,0.444,10
1,Jump Shot,Jump Shot,35,20000012,33.9093,-101,135,-118.3708,1,0,...,2000-10-31,away,POR,3,465,8,18.26,-10.26,0.444,10
2,Jump Shot,Jump Shot,43,20000012,33.8693,138,175,-118.1318,1,0,...,2000-10-31,away,POR,4,412,8,18.26,-10.26,0.444,10
3,Driving Dunk Shot,Dunk,155,20000012,34.0443,0,0,-118.2698,2,0,...,2000-10-31,away,POR,5,379,8,18.26,-10.26,0.444,10
4,Jump Shot,Jump Shot,244,20000012,34.0553,-145,-11,-118.4148,3,0,...,2000-10-31,away,POR,6,572,8,18.26,-10.26,0.444,10


In [6]:
# def convert_matchup(df):
#     df['isHome'] = df.matchup.apply(lambda x: 1 if  x=="home" else 0)
#     df = df.drop(['matchup'], axis=1)
#     return df
# df = convert_matchup(df)

In [7]:
df.head()

,action_type,combined_shot_type,game_event_id,game_id,lat,loc_x,loc_y,lon,period,playoffs,...,game_date,matchup,opponent,shot_id,time_remaining,points,season_avg,plus_minus,FGP,month
0,Jump Shot,Jump Shot,12,20000012,34.0443,-157,0,-118.4268,1,0,...,2000-10-31,away,POR,2,622,8,18.26,-10.26,0.444,10
1,Jump Shot,Jump Shot,35,20000012,33.9093,-101,135,-118.3708,1,0,...,2000-10-31,away,POR,3,465,8,18.26,-10.26,0.444,10
2,Jump Shot,Jump Shot,43,20000012,33.8693,138,175,-118.1318,1,0,...,2000-10-31,away,POR,4,412,8,18.26,-10.26,0.444,10
3,Driving Dunk Shot,Dunk,155,20000012,34.0443,0,0,-118.2698,2,0,...,2000-10-31,away,POR,5,379,8,18.26,-10.26,0.444,10
4,Jump Shot,Jump Shot,244,20000012,34.0553,-145,-11,-118.4148,3,0,...,2000-10-31,away,POR,6,572,8,18.26,-10.26,0.444,10


In [8]:
def encode_and_bind(original_dataframe, feature_to_encode):
    dummies = pd.get_dummies(original_dataframe[[feature_to_encode]])
    res = pd.concat([original_dataframe, dummies], axis=1)
    res = res.drop([feature_to_encode], axis=1)
    return res

In [9]:
def one_hot_encoding(df):
    df_new = df.copy(deep=True)
    features_to_encode = ['combined_shot_type', 'season', 'shot_type', 'shot_zone_area', 'shot_zone_basic', 'shot_zone_range', 'opponent', 'action_type', 'matchup']
    for feature in features_to_encode:
        df_new = encode_and_bind(df_new, feature)

    return df_new

In [10]:
df = one_hot_encoding(df)

In [11]:
df.head()

,game_event_id,game_id,lat,loc_x,loc_y,lon,period,playoffs,shot_distance,shot_made_flag,...,action_type_Step Back Jump shot,action_type_Tip Layup Shot,action_type_Tip Shot,action_type_Turnaround Bank shot,action_type_Turnaround Fadeaway shot,action_type_Turnaround Finger Roll Shot,action_type_Turnaround Hook Shot,action_type_Turnaround Jump Shot,matchup_away,matchup_home
0,12,20000012,34.0443,-157,0,-118.4268,1,0,15,0.0,...,0,0,0,0,0,0,0,0,1,0
1,35,20000012,33.9093,-101,135,-118.3708,1,0,16,1.0,...,0,0,0,0,0,0,0,0,1,0
2,43,20000012,33.8693,138,175,-118.1318,1,0,22,0.0,...,0,0,0,0,0,0,0,0,1,0
3,155,20000012,34.0443,0,0,-118.2698,2,0,0,1.0,...,0,0,0,0,0,0,0,0,1,0
4,244,20000012,34.0553,-145,-11,-118.4148,3,0,14,0.0,...,0,0,0,0,0,0,0,0,1,0


# Splitting the dataset

## Random
### Like this?

In [136]:
def split_random(df, train=None, test=None, validation=None):
    # 80-20
    train_data, rest_data = train_test_split(df, train_size=0.8, shuffle=False)
    # test 10 val 10
    validation_data, test_data = train_test_split(rest_data, test_size=0.5, shuffle=False)

    print(f"Size of training dataset {len(train_data)}" )
    print(f"Size of validation dataset {len(validation_data)}" )
    print(f"Size of test dataset {len(test_data)}" )

    # print(f"Size of train vs test ratio {len(df_train) / (len(df_train) + len(df_test)) }" )
    return train_data, validation_data, test_data

In [137]:
a,b,c = split_random(df)

Size of training dataset 20557
Size of validation dataset 2570
Size of test dataset 2570


## Regular (train), playoff (test)
### Do we need train, test, validation here?

In [133]:
def split_regular_playoff(df, season, train=None, test=None, validation=None):

    # if(train + test + validation != 1):
    #     print("Error dataset split")
    #     return None

    df_validaiton = None

    df_season = df[df['season'] == season]
    isPlayoffs = df_season.playoffs == 1

    df_train = df_season[~isPlayoffs]
    df_test  = df_season[isPlayoffs]

    print(f"Size of training dataset {len(df_train)}" )
    print(f"Size of test dataset {len(df_test)}" )
    print(f"Size of train vs test ratio {len(df_train) / (len(df_train) + len(df_test)) }" )

    return df_train, df_test

In [95]:
a,b = split_regular_playoff(df, '2010-11')

Size of training dataset 1360
Size of test dataset 161
Size of train vs test ratio 0.8941485864562788


## Pick one team, train seasons before, test seasons after
### Should we train on multiple seasons?


In [96]:
df.playoffs.value_counts()

0    21939
1     3758
Name: playoffs, dtype: int64

In [129]:
def split_team_season(df, opponent,  season_train, season_test, train=None, test=None, validation=None):

    # if(train + test + validation != 1):
    #     print("Error dataset split")
    #     return None

    df_train = None
    df_test = None
    df_validaiton = None

    if season_train == [] or season_test == []:
        print("Empty seasons")
        return None

    df_train = df[df['season'].isin(season_train)  & (df['opponent'] == opponent) ]
    df_test = df[(df['season'].isin(season_test)) & (df['opponent'] == opponent) ]
    print(f"Testing vs {opponent}\ntraining: {season_train}\ntesting: {season_test}" )
    print(f"Size of training dataset {len(df_train)}" )
    print(f"Size of test dataset {len(df_test)}" )
    print(f"Size of train vs test ratio {len(df_train) / (len(df_train) + len(df_test)) }" )

    return df_train, df_test

In [134]:
a,b = split_team_season(df, 'POR', ['2000-01'], ['2002-03', '2003-04'] )

Testing vs POR
training: ['2000-01']
testing: ['2002-03', '2003-04']
Size of training dataset 109
Size of test dataset 152
Size of train vs test ratio 0.41762452107279696


In [ ]:
# df.to_csv('../data/data3.csv', index=False)